In [1]:
include(dirname(dirname(pwd()))*"\\src\\TuLiPa.jl");
using .TuLiPa
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS
include(dirname(dirname(pwd())) * raw"\demos\other\flowbased_utils.jl")

remove_cap_for_flowbased_transm (generic function with 1 method)

In [2]:
ptdf_csv_path = raw"X:\Prosjekter\2022_FoU_markedsmodell_julia\per\tulipa_nett\TuLiPa\demos\other\ptdfs_new.csv"
net_data_path = raw"X:\Prosjekter\2022_FoU_markedsmodell_julia\per\tulipa_nett\input\Uke_31\input\nett.json"

"X:\\Prosjekter\\2022_FoU_markedsmodell_julia\\per\\tulipa_nett\\input\\Uke_31\\input\\nett.json"

In [3]:
balances = ["PowerBalance_FINLAND",
"PowerBalance_VESTSYD",
"PowerBalance_HALLINGDAL",
"PowerBalance_TELEMARK",
"PowerBalance_RUSSLAND_EKS",
"PowerBalance_HAUGESUND",
"PowerBalance_VESTMIDT",
"PowerBalance_RUSSLAND_IMP",
"PowerBalance_FINNMARK",
"PowerBalance_DANM-DK1",
"PowerBalance_SVARTISEN",
"PowerBalance_SOROST",
"PowerBalance_SVER-SE2",
"PowerBalance_NORGEMIDT",
"PowerBalance_SORLAND",
"PowerBalance_POLEN",
"PowerBalance_SVER-SE3",
"PowerBalance_MOERE",
"PowerBalance_NEDERLAND",
"PowerBalance_DANM-DK2",
"PowerBalance_OSTLAND",
"PowerBalance_UK",
"PowerBalance_HELGELAND",
"PowerBalance_SVER-SE1",
"PowerBalance_ESTLAND",
"PowerBalance_SVER-SE4",
"PowerBalance_TYSKLAND",
"PowerBalance_TROMS",
"PowerBalance_SVER-SE3A",
"PowerBalance_NORDVEST",
"PowerBalance_LITAUEN"];

## Create modelobjects and solve

In [4]:
function temp_get_data(net_data_path)
    transm = TuLiPa.getelements(JSON.parsefile(net_data_path), net_data_path)

    power_horizon = TuLiPa.SequentialHorizon(2, Day(1))
    push!(transm, TuLiPa.getelement(TuLiPa.COMMODITY_CONCEPT, "BaseCommodity", "Power", 
        (TuLiPa.HORIZON_CONCEPT, power_horizon)))
            
    TuLiPa.addscenariotimeperiod!(transm, "ScenarioTimePeriod", 
        TuLiPa.getisoyearstart(1991), 
        TuLiPa.getisoyearstart(2000))

    return transm
end

temp_get_data (generic function with 1 method)

In [5]:
T = TuLiPa.DataElement

transm  = temp_get_data(net_data_path);
df = CSV.read(ptdf_csv_path, DataFrame)

df = proc_ptdf_csv(df)

transm = remove_cap_for_flowbased_transm(transm, df)    
flow_based = make_flowbased(df, transm)
final_elem = vcat(transm, flow_based)   
final_elem = Vector{T}(final_elem);


e = [TuLiPa.DataElement("Balance", "BaseBalance", b, Dict("Commodity" => "Power")) for b in balances]
final_elem = vcat(e, final_elem);
    
modelobjects = TuLiPa.getmodelobjects(final_elem)

mymodel = JuMP.Model(HiGHS.Optimizer)
prob = TuLiPa.JuMP_Prob(modelobjects, mymodel)
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TuLiPa.TwoTime(datatime, scenariotime)    

update!(prob, prob_time)
solve!(prob)

In [6]:
steps = 1
problem = prob
resultobjects = collect(values(modelobjects))
numperiods_powerhorizon = 1
numperiods_hydrohorizon = 1
periodduration_power = Day(1)
t = 1
includeexogenprice=true

prices, rhstermvalues, production, consumption, hydrolevels, batterylevels, powerbalances, rhsterms, rhstermbalances, plants, plantbalances, plantarrows, demands, demandbalances, demandarrows, hydrostorages, batterystorages  = init_results(steps, problem, modelobjects, resultobjects, numperiods_powerhorizon, numperiods_hydrohorizon, periodduration_power, t, includeexogenprice);